# Deterministic test

In [1]:
from typing import Final, List, Tuple

import numpy as np

from classiq import *

# Hamiltonian
HAMILTONIAN = (
    -0.1 * Pauli.Z(0) * Pauli.Z(1)
    + 0.2 * Pauli.X(0)
    - 0.3 * Pauli.Z(1) * Pauli.Z(2)
    + 0.4 * Pauli.Z(1)
    - 0.1 * Pauli.Y(2) * Pauli.Z(3)
    + 0.2 * Pauli.Y(2)
    - 0.5 * Pauli.X(2) * Pauli.Z(3)
    + 0.2 * Pauli.Z(3)
    - 0.1 * Pauli.Y(1)
    + 0.3 * Pauli.I(0)
)

NUM_QUBITS = 4

# Operator pool
single_x_ops = [Pauli.X(i) for i in range(4)]
single_y_ops = [Pauli.Y(i) for i in range(4)]


# op_pool = single_x_ops + single_y_ops + single_z_ops
op_pool = single_x_ops + single_y_ops


@qfunc
def adapt_layer(idx: int, theta: CReal, qba: QArray):
    suzuki_trotter(op_pool[idx], theta, 1, 1, qba)


@qfunc
def adapt_vqe_ansatz(
    thetas: CArray[CReal],
    ansatz_ops: List[int],
    qba: QArray,
):
    n = thetas.len
    for i in range(n):
        adapt_layer(ansatz_ops_indicies[i], thetas[i], qba)


ansatz_ops_indicies: list[int] = []
# start with default parameter
ansatz_ops_indicies.append(0)
# ansatz state energy
energies = []


# number of layers (initially=1)
k = len(ansatz_ops_indicies)
# trace and history for analysis and plotting
cost_trace = []
params_history = []


@qfunc
def main(params: CArray[CReal, k], v: Output[QArray[QBit, NUM_QUBITS]]):
    allocate(v)
    n = params.len
    # print(f"Building ansatz with {n} layers.")
    adapt_vqe_ansatz(params, ansatz_ops_indicies, v)


qprog = synthesize(main)
execution_preferences = ExecutionPreferences(
    backend_preferences=ClassiqBackendPreferences(
        backend_name=ClassiqSimulatorBackendNames.SIMULATOR_STATEVECTOR
    ),
    random_seed=10,
)
qprog = synthesize(main)

# write an equivalent qmod file
write_qmod(main, name="adapt_vqe", decimal_precision=15, symbolic_only=False)

# initial optimization parameters for the VQE re-optimization
initial_params = np.linspace(0, 1, k).tolist()
with ExecutionSession(qprog, execution_preferences) as es:
    optimization_results = es.minimize(
        cost_function=HAMILTONIAN,  # Hamiltonian problem
        initial_params={"params": initial_params},
        max_iteration=100,
    )

params = optimization_results[-1][1]["params"]
print(f"params: {params}")
energy = optimization_results[-1][0]
energies.append(energy)
print(f"energy = {energy}")

params: [0.0001]
energy = 0.5000000019999999
